In [1]:
import cv2
from numpy import *
import numpy as np
import os
import random

In [2]:
pathR = "../right/"
pathL = "../left/"

pathStR = "../both/right/"
pathStL = "../both/left/"

pathTestR = "../test/right/"
pathTestL = "../test/left/"

calibParams = "./calibParams/"

## Fetching images of individual cam

In [3]:
sizeR = size(os.listdir(pathR))
sizeL = size(os.listdir(pathL))

offset = 9
listingsR = []
for i in range(offset+0, offset+sizeR):
    listingsR.append(f'{i}_R_.png')
#endfor

listingsL = []
for i in range(offset+0, offset+sizeL):
    listingsL.append(f'{i}_L_.png')
#endfor

print(sizeR, sizeL)
print(listingsR, listingsL)

print("\nAfter shuffle:\n")
random.shuffle(listingsR)
random.shuffle(listingsL)
print(listingsR, listingsL)

33 42
['9_R_.png', '10_R_.png', '11_R_.png', '12_R_.png', '13_R_.png', '14_R_.png', '15_R_.png', '16_R_.png', '17_R_.png', '18_R_.png', '19_R_.png', '20_R_.png', '21_R_.png', '22_R_.png', '23_R_.png', '24_R_.png', '25_R_.png', '26_R_.png', '27_R_.png', '28_R_.png', '29_R_.png', '30_R_.png', '31_R_.png', '32_R_.png', '33_R_.png', '34_R_.png', '35_R_.png', '36_R_.png', '37_R_.png', '38_R_.png', '39_R_.png', '40_R_.png', '41_R_.png'] ['9_L_.png', '10_L_.png', '11_L_.png', '12_L_.png', '13_L_.png', '14_L_.png', '15_L_.png', '16_L_.png', '17_L_.png', '18_L_.png', '19_L_.png', '20_L_.png', '21_L_.png', '22_L_.png', '23_L_.png', '24_L_.png', '25_L_.png', '26_L_.png', '27_L_.png', '28_L_.png', '29_L_.png', '30_L_.png', '31_L_.png', '32_L_.png', '33_L_.png', '34_L_.png', '35_L_.png', '36_L_.png', '37_L_.png', '38_L_.png', '39_L_.png', '40_L_.png', '41_L_.png', '42_L_.png', '43_L_.png', '44_L_.png', '45_L_.png', '46_L_.png', '47_L_.png', '48_L_.png', '49_L_.png', '50_L_.png']

After shuffle:

['

In [4]:
imgRList = []
i = 0
while(i<sizeR):
    imR = cv2.imread(pathR+listingsR[i], 0)
    imgRList.append(imR)
    i+=1
#endwhile
print(len(imgRList))

imgLList = []
i = 0
while(i<sizeL):
    imL = cv2.imread(pathL+listingsL[i], 0)
    imgLList.append(imL)
    i+=1
#endwhile
print(len(imgLList))

if len(imgRList) > 0:
    print(imgRList[0].shape)
#endif

if len(imgLList) > 0:
    print(imgLList[0].shape)
#endif

33
42
(1200, 1600)
(1200, 1600)


In [5]:
chessSqSize = 27.3
h, w = 7, 6
patternSize = (h,w)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((w*h,3), np.float32)
objp[:,:2] = np.mgrid[0:h,0:w].T.reshape(-1,2)*chessSqSize
print(objp)

chessCriteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 26, 0.001)
findChessboardFlags = None
downScaleSize = (640,480)
# font
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (50, 50)
# fontScale
fontScale = 1
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2

[[  0.    0.    0. ]
 [ 27.3   0.    0. ]
 [ 54.6   0.    0. ]
 [ 81.9   0.    0. ]
 [109.2   0.    0. ]
 [136.5   0.    0. ]
 [163.8   0.    0. ]
 [  0.   27.3   0. ]
 [ 27.3  27.3   0. ]
 [ 54.6  27.3   0. ]
 [ 81.9  27.3   0. ]
 [109.2  27.3   0. ]
 [136.5  27.3   0. ]
 [163.8  27.3   0. ]
 [  0.   54.6   0. ]
 [ 27.3  54.6   0. ]
 [ 54.6  54.6   0. ]
 [ 81.9  54.6   0. ]
 [109.2  54.6   0. ]
 [136.5  54.6   0. ]
 [163.8  54.6   0. ]
 [  0.   81.9   0. ]
 [ 27.3  81.9   0. ]
 [ 54.6  81.9   0. ]
 [ 81.9  81.9   0. ]
 [109.2  81.9   0. ]
 [136.5  81.9   0. ]
 [163.8  81.9   0. ]
 [  0.  109.2   0. ]
 [ 27.3 109.2   0. ]
 [ 54.6 109.2   0. ]
 [ 81.9 109.2   0. ]
 [109.2 109.2   0. ]
 [136.5 109.2   0. ]
 [163.8 109.2   0. ]
 [  0.  136.5   0. ]
 [ 27.3 136.5   0. ]
 [ 54.6 136.5   0. ]
 [ 81.9 136.5   0. ]
 [109.2 136.5   0. ]
 [136.5 136.5   0. ]
 [163.8 136.5   0. ]]


## Right cam calibration

In [6]:
imgRightPoints = []
objRightPoints = []

j=0
while(j<sizeR):
    print(f'getting: {j}...')
    imageSize = imgRList[j].shape
    rightFound, rightCorners = cv2.findChessboardCorners(imgRList[j], patternSize, flags = findChessboardFlags)
    
    if rightFound:
        cv2.cornerSubPix(imgRList[j], rightCorners, (13,13), (-1,-1), chessCriteria)
        print(f'{str(rightFound)}: {j}')
    #endif

    cv2.drawChessboardCorners(imgRList[j], patternSize, rightCorners, rightFound)
    cv2.putText(imgRList[j], f'{j}/{sizeR}', org, font, fontScale, color, thickness, cv2.LINE_AA)

    if rightFound:
        cv2.putText(imgRList[j], 'Press a to accept, any other key to reject.', (50, 150), font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow(f'r chess', cv2.resize(imgRList[j], downScaleSize))
        if cv2.waitKey(0) & 0xff == ord('a'):
            imgRightPoints.append(rightCorners)
            objRightPoints.append(objp)
            print(f'accepted: {j}')
        else:
            cv2.waitKey(200)
            print(f'rejected: {j}')
        #endif
    else:
        cv2.imshow(f'r chess', cv2.resize(imgRList[j], downScaleSize))
        cv2.waitKey(200)
        print(f'auto rejected: {j}')
    #endif

    print('...........................')

    j+=1
#endwhile
cv2.destroyAllWindows()

getting: 0...
True: 0


QApplication: invalid style override 'adwaita' passed, ignoring it.
	Available styles: Windows, Fusion


accepted: 0
...........................
getting: 1...
True: 1
accepted: 1
...........................
getting: 2...
True: 2
accepted: 2
...........................
getting: 3...
True: 3
accepted: 3
...........................
getting: 4...
True: 4
accepted: 4
...........................
getting: 5...
True: 5
accepted: 5
...........................
getting: 6...
True: 6
accepted: 6
...........................
getting: 7...
True: 7
accepted: 7
...........................
getting: 8...
True: 8
accepted: 8
...........................
getting: 9...
True: 9
accepted: 9
...........................
getting: 10...
True: 10
accepted: 10
...........................
getting: 11...
True: 11
accepted: 11
...........................
getting: 12...
True: 12
accepted: 12
...........................
getting: 13...
True: 13
accepted: 13
...........................
getting: 14...
True: 14
accepted: 14
...........................
getting: 15...
True: 15
accepted: 15
...........................
getting: 16.

In [7]:
cv2.destroyAllWindows()
print(len(imgRightPoints))
print(len(objRightPoints))

31
31


In [8]:
# objRightPoints[0].type()
retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(
    objRightPoints,
    imgRightPoints,
    imgRList[0].shape[::-1],
    None,None
)

hR,wR = imgRList[0].shape[:2]
OmtxR, roiR = cv2.getOptimalNewCameraMatrix(mtxR, distR, (wR,hR), 1, (wR,hR))
print('ret - right: ', retR)
print("roiR: ", roiR)

# https://docs.opencv.org/4.5.4/dc/dbb/tutorial_py_calibration.html
meanRightError = 0
for i in range(len(objRightPoints)):
    imgRightPoints2, _ = cv2.projectPoints(objRightPoints[i], rvecsR[i], tvecsR[i], mtxR, distR)
    error = cv2.norm(imgRightPoints[i], imgRightPoints2, cv2.NORM_L2)/len(imgRightPoints2)
    meanRightError += error
#endfor

print("Total reprojection error: {}".format(meanRightError/len(objRightPoints)))

ret - right:  0.31521250198282913
roiR:  (323, 181, 905, 831)
Total reprojection error: 0.03685525641312678


In [9]:
img = imgRList[25].copy()
# using cv2.undistort
dst = cv2.undistort(img, mtxR, distR, None, OmtxR)
x,y,w,h = roiR
dst = dst[y:y+h, x:x+w]

# using remapping
rightMap = cv2.initUndistortRectifyMap(mtxR, distR, None, OmtxR, img.shape[::-1], cv2.CV_16SC2)
rdst = cv2.remap(img,rightMap[0],rightMap[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
x,y,w,h = roiR
rdst = rdst[y:y+h, x:x+w]

cv2.imshow('undist', cv2.resize(dst, downScaleSize))
cv2.imshow('rundist', cv2.resize(rdst, downScaleSize))
cv2.imshow('img', cv2.resize(img, downScaleSize))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
cv2.destroyAllWindows()

## Left cam calibration

In [11]:
imgLeftPoints = []
objLeftPoints = []

j=0
while(j<sizeL):
    print(f'getting: {j}...')
    imageSize = imgLList[j].shape
    leftFound, leftCorners = cv2.findChessboardCorners(imgLList[j], patternSize, flags = findChessboardFlags)
    
    if leftFound:
        cv2.cornerSubPix(imgLList[j], leftCorners, (11,11), (-1,-1), chessCriteria)
        print(f'{str(leftFound)}: {j}')
    #endif

    cv2.drawChessboardCorners(imgLList[j], patternSize, leftCorners, leftFound)
    cv2.putText(imgLList[j], f'{j}/{sizeL}', org, font, fontScale, color, thickness, cv2.LINE_AA)

    if leftFound:
        cv2.putText(imgLList[j], 'Press a to accept, any other key to reject.', (50, 150), font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow(f'l chess', cv2.resize(imgLList[j], downScaleSize))
        if cv2.waitKey(0) & 0xff == ord('a'):
            imgLeftPoints.append(leftCorners)
            objLeftPoints.append(objp)
            print(f'accepted: {j}')
        else:
            cv2.waitKey(200)
            print(f'rejected: {j}')
        #endif
    else:
        cv2.imshow(f'l chess', cv2.resize(imgLList[j], downScaleSize))
        cv2.waitKey(200)
        print(f'auto rejected: {j}')
    #endif

    print('...........................')

    j+=1
#endwhile
cv2.destroyAllWindows()

getting: 0...
True: 0
accepted: 0
...........................
getting: 1...
True: 1
accepted: 1
...........................
getting: 2...
True: 2
accepted: 2
...........................
getting: 3...
True: 3
accepted: 3
...........................
getting: 4...
True: 4
accepted: 4
...........................
getting: 5...
True: 5
accepted: 5
...........................
getting: 6...
True: 6
accepted: 6
...........................
getting: 7...
True: 7
accepted: 7
...........................
getting: 8...
True: 8
accepted: 8
...........................
getting: 9...
True: 9
accepted: 9
...........................
getting: 10...
True: 10
accepted: 10
...........................
getting: 11...
True: 11
accepted: 11
...........................
getting: 12...
True: 12
accepted: 12
...........................
getting: 13...
True: 13
accepted: 13
...........................
getting: 14...
True: 14
accepted: 14
...........................
getting: 15...
True: 15
accepted: 15
..................

In [12]:
cv2.destroyAllWindows()
print(len(imgLeftPoints))
print(len(objLeftPoints))

40
40


In [13]:
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(
    objLeftPoints,
    imgLeftPoints,
    imgLList[0].shape[::-1],
    None,None
)

hL,wL = imgLList[0].shape[:2]
OmtxL, roiL = cv2.getOptimalNewCameraMatrix(mtxL, distL, (wL,hL), 1, (wL,hL))
print('ret - left: ', retL)
print("roiL: ", roiL)

meanLeftError = 0
for i in range(len(objRightPoints)):
    imgLeftPoints2, _ = cv2.projectPoints(objLeftPoints[i], rvecsL[i], tvecsL[i], mtxL, distL)
    error = cv2.norm(imgLeftPoints[i], imgLeftPoints2, cv2.NORM_L2)/len(imgLeftPoints2)
    meanLeftError += error
print("Total reprojection error: {}".format(meanLeftError/len(objLeftPoints)) )

ret - left:  0.23348144139788274
roiL:  (554, 289, 732, 464)
Total reprojection error: 0.02497199904110289


In [14]:
img = imgLList[1].copy()
# using cv2.undistort
dst = cv2.undistort(img, mtxL, distL, None, OmtxL)
x,y,w,h = roiL
dst = dst[y:y+h, x:x+w]

# using remapping
leftMap = cv2.initUndistortRectifyMap(mtxL, distL, None, OmtxL, img.shape[::-1], cv2.CV_16SC2)
ldst = cv2.remap(img,leftMap[0],leftMap[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
x,y,w,h = roiL
ldst = ldst[y:y+h, x:x+w]

cv2.imshow('undist', cv2.resize(dst, downScaleSize))
cv2.imshow('rundist', cv2.resize(ldst, downScaleSize))
cv2.imshow('img', cv2.resize(img, downScaleSize))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
formatm = [["fx","0","cx"],["0","fy","cy"],["0","0","1"]]
print('\n format -> \n',formatm)
print('\n mtxR -> \n',mtxR,2*'\n','OmtxR -> \n',OmtxR,2*'\n','mtxL -> \n',mtxL,2*'\n','OmtxL -> \n',OmtxL)


 format -> 
 [['fx', '0', 'cx'], ['0', 'fy', 'cy'], ['0', '0', '1']]

 mtxR -> 
 [[875.03979391   0.         795.09083688]
 [  0.         881.90214803 603.69302505]
 [  0.           0.           1.        ]] 

 OmtxR -> 
 [[494.92663574   0.         773.03382807]
 [  0.         610.92681885 599.13994658]
 [  0.           0.           1.        ]] 

 mtxL -> 
 [[883.43490764   0.         810.05715922]
 [  0.         891.76966689 589.48173261]
 [  0.           0.           1.        ]] 

 OmtxL -> 
 [[299.83584595   0.         924.93948095]
 [  0.         296.91784668 516.26671434]
 [  0.           0.           1.        ]]


## Stereo cam calibration

In [16]:
sizeStR = size(os.listdir(pathStR))
sizeStL = size(os.listdir(pathStL))

offset = 9
listingsStR = []
for i in range(offset+0, offset+sizeStR):
    listingsStR.append(f'{i}_R_.png')
#endfor

listingsStL = []
for i in range(offset+0, offset+sizeStL):
    listingsStL.append(f'{i}_L_.png')
#endfor

print(sizeStR, sizeStL)
print(listingsStR, listingsStL)

print("\nAfter shuffle:\n")
temp = list(zip(listingsStR, listingsStL))
random.shuffle(temp)
listingsStR, listingsStL = zip(*temp)
print(listingsStR, listingsStL)

47 47
['9_R_.png', '10_R_.png', '11_R_.png', '12_R_.png', '13_R_.png', '14_R_.png', '15_R_.png', '16_R_.png', '17_R_.png', '18_R_.png', '19_R_.png', '20_R_.png', '21_R_.png', '22_R_.png', '23_R_.png', '24_R_.png', '25_R_.png', '26_R_.png', '27_R_.png', '28_R_.png', '29_R_.png', '30_R_.png', '31_R_.png', '32_R_.png', '33_R_.png', '34_R_.png', '35_R_.png', '36_R_.png', '37_R_.png', '38_R_.png', '39_R_.png', '40_R_.png', '41_R_.png', '42_R_.png', '43_R_.png', '44_R_.png', '45_R_.png', '46_R_.png', '47_R_.png', '48_R_.png', '49_R_.png', '50_R_.png', '51_R_.png', '52_R_.png', '53_R_.png', '54_R_.png', '55_R_.png'] ['9_L_.png', '10_L_.png', '11_L_.png', '12_L_.png', '13_L_.png', '14_L_.png', '15_L_.png', '16_L_.png', '17_L_.png', '18_L_.png', '19_L_.png', '20_L_.png', '21_L_.png', '22_L_.png', '23_L_.png', '24_L_.png', '25_L_.png', '26_L_.png', '27_L_.png', '28_L_.png', '29_L_.png', '30_L_.png', '31_L_.png', '32_L_.png', '33_L_.png', '34_L_.png', '35_L_.png', '36_L_.png', '37_L_.png', '38_L_

In [17]:
imgStRList = []
i = 0
while(i<sizeStR):
    imStR = cv2.imread(pathStR+listingsStR[i], 0)
    imgStRList.append(imStR)
    i+=1
#endwhile
print(len(imgStRList))

imgStLList = []
i = 0
while(i<sizeStL):
    imStL = cv2.imread(pathStL+listingsStL[i], 0)
    imgStLList.append(imStL)
    i+=1
#endwhile
print(len(imgStLList))

print(imgStRList[0].shape)
print(imgStLList[0].shape)

47
47
(1200, 1600)
(1200, 1600)


In [18]:
chessSqSize = 27.3
h, w = 7,6
patternSize = (h,w)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((w*h,3), np.float32)
objp[:,:2] = np.mgrid[0:h,0:w].T.reshape(-1,2)*chessSqSize
print(objp)

chessCriteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 26, 0.001)
findChessboardFlags = None
downScaleSize = (640,480)
# font
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (50, 50)
# fontScale
fontScale = 1
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2

[[  0.    0.    0. ]
 [ 27.3   0.    0. ]
 [ 54.6   0.    0. ]
 [ 81.9   0.    0. ]
 [109.2   0.    0. ]
 [136.5   0.    0. ]
 [163.8   0.    0. ]
 [  0.   27.3   0. ]
 [ 27.3  27.3   0. ]
 [ 54.6  27.3   0. ]
 [ 81.9  27.3   0. ]
 [109.2  27.3   0. ]
 [136.5  27.3   0. ]
 [163.8  27.3   0. ]
 [  0.   54.6   0. ]
 [ 27.3  54.6   0. ]
 [ 54.6  54.6   0. ]
 [ 81.9  54.6   0. ]
 [109.2  54.6   0. ]
 [136.5  54.6   0. ]
 [163.8  54.6   0. ]
 [  0.   81.9   0. ]
 [ 27.3  81.9   0. ]
 [ 54.6  81.9   0. ]
 [ 81.9  81.9   0. ]
 [109.2  81.9   0. ]
 [136.5  81.9   0. ]
 [163.8  81.9   0. ]
 [  0.  109.2   0. ]
 [ 27.3 109.2   0. ]
 [ 54.6 109.2   0. ]
 [ 81.9 109.2   0. ]
 [109.2 109.2   0. ]
 [136.5 109.2   0. ]
 [163.8 109.2   0. ]
 [  0.  136.5   0. ]
 [ 27.3 136.5   0. ]
 [ 54.6 136.5   0. ]
 [ 81.9 136.5   0. ]
 [109.2 136.5   0. ]
 [136.5 136.5   0. ]
 [163.8 136.5   0. ]]


In [19]:
imgStLeftPoints = []
imgStRightPoints = []
objStPoints = []

j=0
while(j<sizeStR):
    print(f'getting: {j}...')
    imageSize = imgStLList[j].shape

    leftFound, leftCorners = cv2.findChessboardCorners(imgStLList[j], patternSize, flags = findChessboardFlags)
    rightFound, rightCorners = cv2.findChessboardCorners(imgStRList[j], patternSize, flags = findChessboardFlags)

    if leftFound:
        cv2.cornerSubPix(imgStLList[j], leftCorners, (11,11), (-1,-1), chessCriteria)
        print(f'Left = {str(leftFound)}: {j}')
    #endif

    if rightFound:
        cv2.cornerSubPix(imgStRList[j], rightCorners, (11,11), (-1,-1), chessCriteria)
        print(f'Right = {str(leftFound)}: {j}')
    #endif

    cv2.drawChessboardCorners(imgStLList[j], patternSize, leftCorners, leftFound)
    cv2.drawChessboardCorners(imgStRList[j], patternSize, rightCorners, rightFound)
    cv2.putText(imgStLList[j], f'{j}/{sizeStL}', org, font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.putText(imgStRList[j], f'{j}/{sizeStR}', org, font, fontScale, color, thickness, cv2.LINE_AA)


    if leftFound and rightFound:
        cv2.putText(imgStLList[j], 'Press a to accept, any other key to reject.', (50, 150), font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(imgStRList[j], 'Press a to accept, any other key to reject.', (50, 150), font, fontScale, color, thickness, cv2.LINE_AA)
        
        cv2.imshow("stl chess", cv2.resize(imgStLList[j], downScaleSize))
        cv2.imshow("str chess", cv2.resize(imgStRList[j], downScaleSize))

        if cv2.waitKey(0) & 0xff == ord('a'):
            imgStLeftPoints.append(leftCorners)
            imgStRightPoints.append(rightCorners)
            objStPoints.append(objp)
            print(f'accepted: {j}')
        else:
            cv2.waitKey(200)
            print(f'rejected: {j}')
        #endif
    else:
        cv2.imshow("stl chess", cv2.resize(imgStLList[j], downScaleSize))
        cv2.imshow("str chess", cv2.resize(imgStRList[j], downScaleSize))
        cv2.waitKey(200)
        print(f'auto rejected: {j}')
    #endif

    print('...........................')

    j+=1
#endwhile

cv2.destroyAllWindows()

getting: 0...
Left = True: 0
Right = True: 0
accepted: 0
...........................
getting: 1...
Left = True: 1
Right = True: 1
accepted: 1
...........................
getting: 2...
Right = False: 2
auto rejected: 2
...........................
getting: 3...
Left = True: 3
Right = True: 3
accepted: 3
...........................
getting: 4...
Left = True: 4
Right = True: 4
accepted: 4
...........................
getting: 5...
Left = True: 5
Right = True: 5
accepted: 5
...........................
getting: 6...
Right = False: 6
auto rejected: 6
...........................
getting: 7...
Left = True: 7
Right = True: 7
accepted: 7
...........................
getting: 8...
Left = True: 8
Right = True: 8
accepted: 8
...........................
getting: 9...
Right = False: 9
auto rejected: 9
...........................
getting: 10...
Left = True: 10
Right = True: 10
accepted: 10
...........................
getting: 11...
Left = True: 11
Right = True: 11
accepted: 11
...........................

In [20]:
cv2.destroyAllWindows()

In [21]:
print(len(imgStLeftPoints))
print(len(imgStRightPoints))

38
38


In [22]:
# stCriteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
stCriteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
stFlags = cv2.CALIB_FIX_ASPECT_RATIO | cv2.CALIB_ZERO_TANGENT_DIST | cv2.CALIB_SAME_FOCAL_LENGTH | cv2.CALIB_RATIONAL_MODEL | cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5
# stFlags = cv2.CALIB_FIX_INTRINSIC
retS, MLS, dLS, MRS, dRS, R, T, E, F = cv2.stereoCalibrate(
    objStPoints,
    imgStLeftPoints,
    imgStRightPoints,
    mtxL,
    distL,
    mtxR,
    distR,
    imStR.shape[::-1],
    stCriteria,
    stFlags
)

print('ret - stereo ', retS)

print(' MRS -> ',MRS,2*'\n','mtxR -> ',mtxR,2*'\n','MLS -> ',MLS,2*'\n','mtxL -> ',mtxL)
print(' Rotational -> \n',R,2*'\n','Translational -> \n',T,2*'\n','Essential -> \n',E,2*'\n','Fundamental -> \n',F)

ret - stereo  1.4644855359469062
 MRS ->  [[875.03979391   0.         795.09083688]
 [  0.         881.90214803 603.69302505]
 [  0.           0.           1.        ]] 

 mtxR ->  [[875.03979391   0.         795.09083688]
 [  0.         881.90214803 603.69302505]
 [  0.           0.           1.        ]] 

 MLS ->  [[883.43490764   0.         810.05715922]
 [  0.         891.76966689 589.48173261]
 [  0.           0.           1.        ]] 

 mtxL ->  [[883.43490764   0.         810.05715922]
 [  0.         891.76966689 589.48173261]
 [  0.           0.           1.        ]]
 Rotational -> 
 [[ 0.99997159 -0.00420376  0.006257  ]
 [ 0.00448242  0.99896754 -0.04520805]
 [-0.0060605   0.04523481  0.998958  ]] 

 Translational -> 
 [[311.38920352]
 [  0.89643207]
 [-11.42238266]] 

 Essential -> 
 [[ 4.57670387e-02  1.14511394e+01  3.79114330e-01]
 [-9.53488442e+00 -1.40376155e+01 -3.11136205e+02]
 [ 4.99369151e-01  3.11071474e+02 -1.40829080e+01]] 

 Fundamental -> 
 [[-4.57233960e-09

In [ ]:
np.save(calibParams+'R.npy', R)
np.save(calibParams+'T.npy', T)
np.save(calibParams+'E.npy', E)
np.save(calibParams+'F.npy', F)

np.save(calibParams+'MLS.npy', MLS)
np.save(calibParams+'dLS.npy', dLS)
np.save(calibParams+'MRS.npy', MRS)
np.save(calibParams+'dRS.npy', dRS)

In [ ]:
rectifyScale = 0 # if 0 image croped, if 1 image nor croped
RL, RR, PL, PR, Q, roiL, roiR = cv2.stereoRectify(MLS, dLS, MRS, dRS, imStR.shape[::-1], R, T, rectifyScale) # last paramater is alpha, if 0= croped, if 1= not croped
# initUndistortRectifyMap function
leftStereoMap = cv2.initUndistortRectifyMap(MLS, dLS, RL, PL, imStL.shape[::-1], cv2.CV_16SC2)   # cv2.CV_16SC2 this format enables us the programme to work faster
rightStereoMap = cv2.initUndistortRectifyMap(MRS, dRS, RR, PR, imStR.shape[::-1], cv2.CV_16SC2)

In [ ]:
np.save(calibParams+'RL.npy', RL)
np.save(calibParams+'RR.npy', RR)
np.save(calibParams+'PL.npy', PL)
np.save(calibParams+'PR.npy', PR)
np.save(calibParams+'Q.npy', Q)

In [ ]:
def drawLines(img):
    y,count = 0,0
    while(count<40):
        img = cv2.line(img, (0,50+y), (img.shape[1],50+y), (y,count*2,y), 2)
        y+=30
        count+=1
    #endwhile
    return cv2.resize(img, (640,480))
#enddef

In [ ]:
picNum = '9'
rectPath = "../rectified/"
frameL = cv2.imread(pathTestL+picNum+'_L_.png')
frameR = cv2.imread(pathTestR+picNum+'_R_.png')

leftNice = cv2.remap(frameL,leftStereoMap[0],leftStereoMap[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)  # Rectify the image using the kalibration parameters founds during the initialisation
rightNice = cv2.remap(frameR,rightStereoMap[0],rightStereoMap[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)

frameLCpy = frameL.copy()
frameRCpy = frameR.copy()

xr,yr,wr,hr = roiR
xl,yl,wl,hl = roiL

rightImgRemapCp = rightNice.copy()
leftImgRemapCp = leftNice.copy()

cv2.imshow('R', drawLines(frameRCpy))
cv2.imshow('L', drawLines(frameLCpy))
cv2.imshow('R rect', drawLines(rightImgRemapCp))
cv2.imshow('L rect', drawLines(leftImgRemapCp))
cv2.waitKey(0)

origLR = np.concatenate((frameLCpy, frameRCpy), axis=1)
cv2.imwrite(rectPath+picNum+'_orig.png', origLR)
rectLR = np.concatenate((leftImgRemapCp, rightImgRemapCp), axis=1)
cv2.imwrite(rectPath+picNum+'_rect.png', rectLR)
cv2.destroyAllWindows()